# LOAD DATA

In [ ]:
import pandas as pd
import numpy as np
import tqdm

In [ ]:
import numpy as np
import dask.dataframe as dd
import tqdm
from pyspark.sql.window import Window
from sklearn.preprocessing import LabelEncoder
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark import SparkContext
from graphframes import GraphFrame
from pyspark.sql.types import *
import multiprocessing
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation
import seaborn as sns
import matplotlib.pyplot as plt
from pyspark.sql import functions as F

In [ ]:
df = pd.read_csv("../dataset/HI-Small_Trans.csv")

In [ ]:
spark_driver_memory = "8g"
spark_executor_memory = "4g"


spark = SparkSession.builder \
                    .config("spark.driver.memory", spark_driver_memory) \
                    .config("spark.executor.memory", spark_executor_memory) \
                    .master("local[*]") \
                    .getOrCreate()
print("Spark session created")
sc = spark.sparkContext
print("Spark context created")

In [ ]:
spark_df = spark.read.parquet('../project/df.parquet').drop("__index_level_0__")
spark_df = spark_df.withColumn("timestamp", date_format("timestamp", "yyyyMMddHHmm").cast(LongType()))
spark_df.show(5)

In [ ]:
spark_df.persist()
spark_df.count()

### Split train and validation

In [ ]:
#Split the dataset train and test
sorted_df = spark_df.orderBy("timestamp")

# Calcola il numero di righe per l'addestramento e la convalida
total_rows = sorted_df.count()
train_percentage = 0.8
train_rows = int(total_rows * train_percentage)
validation_rows = total_rows - train_rows

# Dividi il DataFrame in modo sequenziale 80/20
train_df = sorted_df.limit(train_rows)
validation_df = sorted_df.subtract(train_df)

# Assegna un indice univoco a ciascuna riga
train_df = train_df.repartition(1)
train_df = train_df.withColumn("index", F.monotonically_increasing_id())
train_df = train_df.repartition(12)
validation_df = validation_df.repartition(1)
validation_df = validation_df.withColumn("index", F.monotonically_increasing_id())
validation_df = validation_df.repartition(12)

# Stampa il numero di righe in ciascun DataFrame
#print("Number of records in training DataFrame: ", train_df.count())
#print("Number of records in validation DataFrame: ", validation_df.count())

In [ ]:
def write_and_load_dataframe(df, name):
    df.write\
    .format("csv")\
    .option("header", "true")\
    .mode("overwrite")\
    .save(f"./preprocessed_data/{name}_small")

    return df

### Find transactions that occur one after the other with the same amount and currency.

In [ ]:
combinations = {}
values = list(sorted_df.select("payment_format").distinct().collect())
j = 0
for i in range(len(values)):
    for k in range(0, len(values)):
        combinations[(i,k)] = j
        j+=1


In [ ]:
def add_money_send_to_send(df):

    train_vertices = df.select(F.col("from_account").alias("id")).union(df.select(F.col("to_account").alias("id"))).distinct()
    train_edges = df.select(F.col("from_account").alias("src"), F.col("to_account").alias("dst"), F.col("index"), F.col("amount_paid").alias("amount"), F.col("timestamp"), F.col("payment_format"), F.col("is_laundering"))
    g = GraphFrame(train_vertices, train_edges)


    schema = StructType([
        StructField("index", LongType(), False),
        StructField("timestamp", DoubleType(), False),
        StructField("from", IntegerType(), False),
        StructField("to", IntegerType(), False),
        StructField("payment_format", IntegerType(), False),
        StructField("is_laundering", IntegerType(), False),
        StructField("payment_payment", IntegerType(), False)
    ])


    motif = "(a)-[c1]->(b); (b)-[c2]->(c)"
    filter_string = "a != b and b != c and c1.amount == c2.amount and c1.timestamp < c2.timestamp"
    graph = g.find(motif).filter(filter_string).distinct()
    graph.cache()
    columns = ['c1', 'c2']
    pattern = np.array(graph.select(*columns).collect()).squeeze()
    total_rows = []

    for row in pattern:
        rows_to_append = []
        payment_formats = []
        if isinstance(row[1], np.ndarray):
            for r in row:
                #index | timestamp | from | to | payment_format | is_laundering 
                rows_to_append.append([int(r[2]), float(r[4]), int(r[0]), int(r[1]), int(r[5]), int(r[6])])
                payment_formats.append(int(r[5]))
        else:
                rows_to_append.append([int(r[2]), float(r[4]), int(r[0]), int(r[1]), int(r[5]), int(r[6])])
                payment_formats.append(int(r[5]))
        
        for r in rows_to_append:
            r.append(combinations[(payment_formats[0], payment_formats[1])])
            total_rows.append(r)

    temp_df = spark.createDataFrame(total_rows, schema)

    temp_df = temp_df.dropDuplicates(['index'])

    joined_df = df.join(temp_df.select("index", "payment_payment").withColumnRenamed("payment_payment", "payment_payment_B"), on="index", how="left")

    # Aggiungi la colonna "payment_payment" a dfA, usando il valore corrispondente da dfB se presente, altrimenti imposta -1
    df = joined_df.withColumn("payment_payment", F.when(F.col("payment_payment_B").isNotNull(), F.col("payment_payment_B")).otherwise(-1)).drop("payment_payment_B")
  
    return df

In [ ]:
train_df = add_money_send_to_send(train_df)
validation_df = add_money_send_to_send(validation_df)

### Find circular patterns

In [ ]:
def find_cycles(df):

    schema = StructType([
        StructField("index", LongType(), False),
        StructField("timestamp", DoubleType(), False),
        StructField("from", IntegerType(), False),
        StructField("to", IntegerType(), False),
        StructField("payment_format", IntegerType(), False),
        StructField("is_laundering", IntegerType(), False),
        StructField("hop_2", IntegerType(), False),
        StructField("hop_3", IntegerType(), False),
        StructField("hop_4", IntegerType(), False),
        StructField("hop_5", IntegerType(), False),
        StructField("hop_6", IntegerType(), False),
        StructField("hop_7", IntegerType(), False),
        StructField("hop_8", IntegerType(), False),
        StructField("hop_9", IntegerType(), False),
        StructField("hop_10", IntegerType(), False),
        StructField("hop_11", IntegerType(), False),
        StructField("hop_12", IntegerType(), False),
        StructField("hop_13", IntegerType(), False),
    ])



    all_df = []
    filtered_spark = df.filter(F.col("payment_currency") == F.col("receiving_currency"))
    filtered_spark.cache()
    #payment_formats = filtered_spark.select("payment_format").distinct().rdd.flatMap(lambda x: x).collect()

    for j in range(1):
        verteces = filtered_spark.filter(F.col("payment_format") == j).select(F.col("from_account").alias("id")).union(spark_df.select(F.col("to_account").alias("id"))).distinct()
        edges = filtered_spark.filter(F.col("payment_format") == j).select(F.col("from_account").alias("src"), F.col("to_account").alias("dst"), F.col("index"), F.col("amount_paid").alias("amount"), F.col("timestamp"), F.col("payment_format"), F.col("is_laundering"))
        g = GraphFrame(verteces, edges)
        g = g.dropIsolatedVertices()
        g.cache()
        for hop in tqdm.tqdm(range(2,15)):
            motif = ""

            for i in range(hop):
                motif += "(n" + str(i) + ")-[c" + str(i+1) + "]->(n" + str((i+1) % hop) + "); "
            motif = motif.strip("; ")

            filter_string = ""
            for i in range(hop):
                for j in range(i, hop-1):
                    filter_string += "n{} != n{}".format(i, j+1)
                    if i+1 < hop-1:
                        filter_string += " and "
            filter_string += " and "
            for j in range(1,hop):
                    filter_string += "c{}.timestamp < c{}.timestamp".format(j, j+1)
                    if(j+1 < hop):
                        filter_string += " and "    
            graph = g.find(motif)
            graph = graph.filter(filter_string)
            select_col = []
            for i in range(hop):
                select_col.append("c{}".format(i+1))
            pattern = np.array(graph.select(*select_col).collect()).squeeze()
            total_rows = []

            for row in pattern:
                if isinstance(row[1], np.ndarray):
                    for r in row:
                        #index | timestamp | from | to | payment_format | is_laundering | hop
                        total_rows.append([int(r[2]), r[4], int(r[0]), int(r[1]), int(r[5]), int(r[6]), hop])
                else:
                    total_rows.append([int(row[2]), row[4], int(row[0]), int(row[1]), int(row[5]), int(row[6]), hop])

            dataframe = pd.DataFrame(total_rows, columns=['index', 'timestamp', 'from', 'to', 'payment_format', 'is_laundering', 'hop'])

            all_df.append(dataframe.drop_duplicates())

    merged_df = pd.concat(all_df, ignore_index=True)
    one_hot_encoded_df = pd.get_dummies(merged_df, columns=['hop'], prefix='hop')

    # Manually add missing hop columns (hop_2 to hop_13) and fill with False
    columns_to_add = [f"hop_{i}" for i in range(2, 14)]
    for col in columns_to_add:
        if col not in one_hot_encoded_df.columns:
            one_hot_encoded_df[col] = False

    grouped_df = one_hot_encoded_df.groupby('index').agg({
        'timestamp': 'first',
        'from': 'first',
        'to': 'first',
        'payment_format': 'first',
        'is_laundering': 'first',
        **{col: 'any' for col in columns_to_add}
    }).reset_index()
   
    columns_to_encode = ['hop_2', 'hop_3', 'hop_4', 'hop_5', 'hop_6', 'hop_7', 'hop_8', 'hop_9', 'hop_10', 'hop_11', 'hop_12', 'hop_13']
    grouped_df[columns_to_encode] = grouped_df[columns_to_encode].fillna(False, inplace=False).astype(int)
    
    temp_df = spark.createDataFrame(grouped_df, schema)

    temp_df = temp_df.dropDuplicates(['index'])

    # Step 1: Seleziona solo le colonne necessarie da temp_df e rinomina le colonne
    temp_df_selected = temp_df.select(
        "index",
        "hop_2", "hop_3", "hop_4", "hop_5", "hop_6",
        "hop_7", "hop_8", "hop_9", "hop_10", "hop_11",
        "hop_12", "hop_13"
    ).withColumnRenamed("hop_2", "hop_2_B").withColumnRenamed("hop_3", "hop_3_B").withColumnRenamed("hop_4", "hop_4_B").withColumnRenamed("hop_5", "hop_5_B").withColumnRenamed("hop_6", "hop_6_B").withColumnRenamed("hop_7", "hop_7_B").withColumnRenamed("hop_8", "hop_8_B").withColumnRenamed("hop_9", "hop_9_B").withColumnRenamed("hop_10", "hop_10_B").withColumnRenamed("hop_11", "hop_11_B").withColumnRenamed("hop_12", "hop_12_B").withColumnRenamed("hop_13", "hop_13_B")

    # Step 2: Esegui una left join tra df e temp_df_selected, usando l'indice come chiave di join
    joined_df = df.join(temp_df_selected, on="index", how="left")

    # Step 3: Usa la funzione when per assegnare il valore corrispondente da hop_i_B se presente, altrimenti imposta il valore a 0
    for i in range(2, 14):
        joined_df = joined_df.withColumn(
            f"hop_{i}", 
            F.when(F.col(f"hop_{i}_B").isNotNull(), F.col(f"hop_{i}_B")).otherwise(0)
        )

    # Step 4: Rimuovi le colonne aggiunte da temp_df_selected
    joined_df = joined_df.drop(*[f"hop_{i}_B" for i in range(2, 14)])

    # Il risultato finale è il dataframe df con le colonne hop_2 a hop_13 aggiunte e i valori 0 dove necessario
    df_result = joined_df

    return df_result

In [ ]:
train_df = find_cycles(train_df)
validation_df = find_cycles(validation_df)

### Find Fan in degree

In [ ]:
def find_fanin(g: GraphFrame):
    motif = "(a)-[c1]->(b); (c)-[c2]->(b)"
    filter_motif = "(abs(c1.timestamp - c2.timestamp)) <= 40000 and c1.index != c2.index and c1.payment_currency == c2.payment_currency"#and c1.payment_format == c2.payment_format"
  
    pattern = g.find(motif).filter(filter_motif).select("c1", "c2").distinct()
    fan_in_trans = pattern.groupBy(F.col("c1")).agg(F.count("*").alias("fan_in_degree")).select(F.col("c1").alias("transaction"), F.col("fan_in_degree"))
    #fan_in_trans.cache()
    return fan_in_trans

def add_fan_in(df):
    filtered_spark = df.filter(F.col("payment_currency") == F.col("receiving_currency"))
    filtered_spark.cache()
    payment_formats = filtered_spark.select("payment_format").distinct().rdd.flatMap(lambda x: x).collect()

    total_fan_in = None

    for payment_format in payment_formats:
        print(f"Find fan in payment_format: {payment_format}")
        filtered_by_format = filtered_spark.filter(F.col("payment_format") == payment_format)
        verteces = (
            filtered_by_format.select(F.col("from_account").alias("id"))
            .union(spark_df.select(F.col("to_account").alias("id")))
            .distinct()
        )
        edges = (
            filtered_by_format.select(
                F.col("from_account").alias("src"),
                F.col("to_account").alias("dst"),
                F.col("index"),
                F.col("timestamp"),
                F.col("payment_currency"),
                F.col("payment_format"),
                F.col("is_laundering")
                
            )
        )
        g = GraphFrame(verteces, edges)
        if total_fan_in is None:
            total_fan_in = find_fanin(g)
        else:
            total_fan_in = total_fan_in.unionAll(find_fanin(g))    
    
    
    def extract_values(transaction):
        src, dst, index, timestamp, payment_currency, payment_format, is_laundering = transaction
        return (src, dst, index, timestamp,payment_currency,  payment_format, is_laundering)

    # Definisci lo schema per il DataFrame Spark
    schema_udf = StructType([
        StructField("src", IntegerType(), True),
        StructField("dst", IntegerType(), True),
        StructField("index", IntegerType(), True),
        StructField("timestamp", FloatType(), True),
        StructField("payment_currency", IntegerType(), True),
        StructField("payment_format", IntegerType(), True),
        StructField("is_laundering", IntegerType(), True)
    ])

    # Applica la funzione UDF per estrarre i valori dalla colonna "transaction" e crea un nuovo DataFrame
    extract_udf = F.udf(extract_values, schema_udf)
    new_spark_df = total_fan_in.withColumn("extracted", extract_udf("transaction"))

    # Seleziona le colonne necessarie e converte il DataFrame Spark in un DataFrame Pandas
    temp_df = new_spark_df.select("extracted.*", "fan_in_degree")

    joined_df = df.join(temp_df.select("index", "fan_in_degree").withColumnRenamed("fan_in_degree", "fan_in_degree_B"), on="index", how="left")

    # Aggiungi la colonna "payment_payment" a dfA, usando il valore corrispondente da dfB se presente, altrimenti imposta -1
    df = joined_df.withColumn("fan_in_degree", F.when(F.col("fan_in_degree_B").isNotNull(), F.col("fan_in_degree_B")).otherwise(0)).drop("fan_in_degree_B")
        
    return df

In [ ]:
train_df = add_fan_in(train_df)
validation_df = add_fan_in(validation_df)

### Find Fan out degree

In [ ]:
def find_fanout(g: GraphFrame):
    motif = "(a)-[c1]->(b); (a)-[c2]->(c)"
    filter_motif = "(abs(c1.timestamp - c2.timestamp)) <= 40000 and c1.index != c2.index"
  
    pattern = g.find(motif).filter(filter_motif).select("c1", "c2").distinct()
    fan_out_trans = pattern.groupBy(F.col("c1")).agg(F.count("*").alias("fan_out_degree")).select(F.col("c1").alias("transaction"), F.col("fan_out_degree"))
    fan_out_trans.cache()
    
    return fan_out_trans

def add_fan_out(df):
    filtered_spark = df.filter(F.col("payment_currency") == F.col("receiving_currency"))
    filtered_spark.cache()
    #payment_formats = filtered_spark.select("payment_format").distinct().rdd.flatMap(lambda x: x).collect()

    total_fan_out = None

    for payment_format in range(2):
        print(f"Find fan in payment_format: {payment_format}")
        filtered_by_format = filtered_spark.filter(F.col("payment_format") == payment_format)
        verteces = (
            filtered_by_format.select(F.col("from_account").alias("id"))
            .union(spark_df.select(F.col("to_account").alias("id")))
            .distinct()
        )
        edges = (
            filtered_by_format.select(
                F.col("from_account").alias("src"),
                F.col("to_account").alias("dst"),
                F.col("index"),
                F.col("timestamp"),
                F.col("payment_format"),
                F.col("is_laundering")
            )
        )
        g = GraphFrame(verteces, edges)
        if total_fan_out is None:
            total_fan_out = find_fanout(g)
        else:
            total_fan_out = total_fan_out.unionAll(find_fanout(g))    
    
    
    def extract_values(transaction):
        src, dst, index, timestamp,  payment_format, is_laundering = transaction
        return (src, dst, index, timestamp, payment_format, is_laundering)

    # Definisci lo schema per il DataFrame Spark
    schema_udf = StructType([
        StructField("src", IntegerType(), True),
        StructField("dst", IntegerType(), True),
        StructField("index", IntegerType(), True),
        StructField("timestamp", FloatType(), True),
        StructField("payment_format", IntegerType(), True),
        StructField("is_laundering", IntegerType(), True)
    ])

    # Applica la funzione UDF per estrarre i valori dalla colonna "transaction" e crea un nuovo DataFrame
    extract_udf = F.udf(extract_values, schema_udf)
    new_spark_df = total_fan_out.withColumn("extracted", extract_udf("transaction"))

    # Seleziona le colonne necessarie e converte il DataFrame Spark in un DataFrame Pandas
    temp_df = new_spark_df.select("extracted.*", "fan_out_degree")

    joined_df = df.join(temp_df.select("index", "fan_out_degree").withColumnRenamed("fan_out_degree", "fan_out_degree_B"), on="index", how="left")

    # Aggiungi la colonna "payment_payment" a dfA, usando il valore corrispondente da dfB se presente, altrimenti imposta -1
    df = joined_df.withColumn("fan_out_degree", F.when(F.col("fan_out_degree_B").isNotNull(), F.col("fan_out_degree_B")).otherwise(0)).drop("fan_out_degree_B")
        
    return df

In [ ]:
train_df = add_fan_out(train_df)
validation_df = add_fan_out(validation_df)

train_df = write_and_load_dataframe(train_df, "train")
validation_df = write_and_load_dataframe(validation_df, "validation")

In [ ]:
def create_final_dataframe(df):
    return df\
    .withColumn("same_account", F.when(F.col("from_account") == F.col("to_account"), 1).otherwise(0))\
    .withColumn("same_currency", F.when(F.col("receiving_currency") == F.col("payment_currency"), 1).otherwise(0))\
    .withColumn("same_bank", F.when(F.col("from_bank") == F.col("to_bank"), 1).otherwise(0))\
    .withColumn("same_amount", F.when(F.col("amount_received") == F.col("amount_paid"), 1).otherwise(0))\
    .drop(F.col("index"))\
    .drop(F.col("timestamp"))\
    .drop(F.col("from_bank"))\
    .drop(F.col("to_bank"))\
    .drop(F.col("from_account"))\
    .drop(F.col("to_account"))

In [ ]:
train_df = create_final_dataframe(train_df)
validation_df = create_final_dataframe(validation_df)

In [ ]:
train_df = train_df.drop("from_account").drop("to_account")
validation_df = validation_df.drop("from_account").drop("to_account")

In [ ]:
train_df = write_and_load_dataframe(train_df, "train")
validation_df = write_and_load_dataframe(validation_df, "validation")